In [16]:
from rmgpy.species import Species
from rmgpy.molecule import Molecule
from rmgpy.molecule.resonance import *
from rmgpy.rmg.react import *
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.data.rmg import getDB

In [17]:


database = RMGDatabase()
database.load(
    path = settings['database.directory'], 
    thermoLibraries = ['primaryThermoLibrary'], # can add others if necessary
    kineticsFamilies = 'default', 
    kineticsDepositories = 'all',
)

In [18]:
SO2 = Species().fromSMILES('O=S=O')
SO2.molecule = generateResonanceStructures(Molecule(SMILES='O=S=O'))
speciesTuple = (SO2,SO2)
reactions = reactSpecies(speciesTuple)

AttributeError: 'NoneType' object has no attribute 'families'

In [ ]:
(Species(index=1, label="SO2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.14742,0.00630357,-2.32432e-06,-1.47053e-09,9.08586e-13,-36923.4,9.44749], Tmin=(100,'K'), Tmax=(1104.84,'K')), NASAPolynomial(coeffs=[4.96626,0.00261551,-1.25026e-06,2.54618e-10,-1.87852e-14,-37502.1,-0.308899], Tmin=(1104.84,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), E0=(-307.114,'kJ/mol'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(62.3585,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(S4dd-OdOd) + group(O2d-S2d) + group(O2d-S2d)"""), conformer=Conformer(E0=(-307.114,'kJ/mol'), modes=[]), molecule=[Molecule(SMILES="O=S=O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="OSO"), Molecule(SMILES="OSO"), Molecule(SMILES="O#SO"), Molecule(SMILES="O#SO")], molecularWeight=(64.0638,'amu'), energyTransferModel=SingleExponentialDown(alpha0=(3.5886,'kJ/mol'), T0=(300,'K'), n=0.85)), Species(index=1, label="SO2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.14742,0.00630357,-2.32432e-06,-1.47053e-09,9.08586e-13,-36923.4,9.44749], Tmin=(100,'K'), Tmax=(1104.84,'K')), NASAPolynomial(coeffs=[4.96626,0.00261551,-1.25026e-06,2.54618e-10,-1.87852e-14,-37502.1,-0.308899], Tmin=(1104.84,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), E0=(-307.114,'kJ/mol'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(62.3585,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(S4dd-OdOd) + group(O2d-S2d) + group(O2d-S2d)"""), conformer=Conformer(E0=(-307.114,'kJ/mol'), modes=[]), molecule=[Molecule(SMILES="O=S=O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="O=[SH]O"), Molecule(SMILES="OSO"), Molecule(SMILES="OSO"), Molecule(SMILES="O#SO"), Molecule(SMILES="O#SO")], molecularWeight=(64.0638,'amu'), energyTransferModel=SingleExponentialDown(alpha0=(3.5886,'kJ/mol'), T0=(300,'K'), n=0.85)))


In [21]:
l = ['v','v','ds']
set(l)
print len(l)
print len(set(l))

3
2


In [ ]:
from IPython.display import display

# Pathfinder

def find_allyl_delocalization_paths(atom1):
    """
    Find all the delocalization paths allyl to the radical center indicated
    by `atom1`. Used to generate resonance isomers.
    """
#     cython.declare(paths=list, atom2=Atom, atom3=Atom, bond12=Bond, bond23=Bond)
    
    # No paths if atom1 is not a radical
    if atom1.radicalElectrons <= 0:
        return []

    # Find all delocalization paths
    paths = []
    for atom2, bond12 in atom1.edges.items():
        # Vinyl bond must be capable of gaining an order
        if (bond12.isSingle() or bond12.isDouble()) and (atom1.radicalElectrons in [1, 2]):
            for atom3, bond23 in atom2.edges.items():
                # Allyl bond must be capable of losing an order without breaking
                if atom1 is not atom3 and (bond23.isDouble() or bond23.isTriple()):
                    paths.append([atom1, atom2, atom3, bond12, bond23])
#     print "allyl paths: {0}".format(paths)
    return paths


def find_lone_pair_radical_delocalization_paths(atom1):
    """
    Find all the delocalization paths of lone electron pairs next to the radical center indicated
    by `atom1`. Used to generate resonance isomers in adjacent N/O/S atoms.
    Two adjacent O atoms are not allowed since (a) currently RMG has no good thermo/kinetics for R[:O+.][:::O-] which
    could have been generated as a resonance structure of R[::O][::O.].
    The radical site (atom1) could be either:
    - `N u1 p0`, eg O=[N.][:::O-]
    - `N u1 p1`, eg R[:NH][:NH.]
    - `O u1 p2`, eg O=N[::O.]; p1 or p2 are not allowed; not allowed when adjacent to another O atom
    - `S u1 p0`, eg O[S.+]([O-])=O
    - `S u1 p1`, eg O[:S.+][O-]
    - `S u1 p2`, eg O=N[::S.]
    - any of the above with more than 1 radical where possible
    The non-radical site (atom2) could respectively be:
    - `N u0 p1`
    - `N u0 p2`
    - `O u0 p3`
    - `S u0 p1`
    - `S u0 p2`
    - `S u0 p3`
    (where ':' denotes a lone pair, '.' denotes a radical, '-' not in [] denotes a single bond, '-'/'+' denote charge)
    """
#     cython.declare(paths=list, atom2=Atom, bond12=Bond)

    paths = []
    if (atom1.isNitrogen() and atom1.radicalElectrons >= 1 and atom1.lonePairs in [0, 1]
            or atom1.isSulfur() and atom1.radicalElectrons >= 1 and atom1.lonePairs in [0, 1, 2]):
        for atom2, bond12 in atom1.edges.items():
            if ((atom2.isNitrogen() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2])
                    or (atom2.isOxygen() and atom2.radicalElectrons == 0 and atom2.lonePairs == 3)
                    or (atom2.isSulfur() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2, 3])
                    and bond12.isSingle()):
                paths.append([atom1, atom2])
    # O is treated separately to avoid RO[::O.] <-> R[:O.+][:::O-], see RMG-Py #1223
    elif atom1.isOxygen() and atom1.radicalElectrons >= 1 and atom1.lonePairs == 2:
        for atom2, bond12 in atom1.edges.items():
            if ((atom2.isNitrogen() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2])
                    or (atom2.isSulfur() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2, 3])
                    and bond12.isSingle()):
                paths.append([atom1, atom2])
#     print "LP-rad paths: {0}".format(paths)
    return paths


def find_lone_pair_multiple_bond_delocalization_paths(atom1):
    """
    Find all the delocalization paths of a N/O/S atom which either:
    - Has a lonePair and is bonded by a single/double bond (e.g., [::N]-[.CH2], [::NH-]-[CH2+], [::N-]=[CH+])
    - Can obtain a lonePair and is bonded by a double/triple bond (e.g., [:NH]=[CH2], [:N]#[CH], [:N.]=[CH2])
    - Can undergo two subsequent transformations as above (important in case the middle structure is incorrect (i.e., we
      purposely don't have atomTypes defined for it, commonly N5td or N5sc/N5dc with +2 charge)
    Giving the following resonance transitions, for example:
    - [:N.]=[CH2] <=> [::N]-[.CH2]
    - [:NH]=[CH2] <=> [::NH-]-[CH2+]
    - [:N]#[CH] <=> [::N-]=[CH+]
    - N#[N+][O-] <=> <=> [N-]=[N+]=O
    - C[N+](=O)[NH-] <=> <=> C[N+]([O-])=[NH]
    - other examples: S#N, N#[S], O=S([O])=O, [NH]=[N+]=[N-]
    Direction "1" is the direction <increasing> the multiple bond as in [::N]-[.CH2] <=> [:N.]=[CH2]
    Direction "2" is the direction <decreasing> the multiple bond as in [:N.]=[CH2] <=> [::N]-[.CH2]
    (where ':' denotes a lone pair, '.' denotes a radical, '-' not in [] denotes a single bond, '-'/'+' denote charge)
    Here we don't distinguish between the radical/non-radical cases, these are dealt in the calling function.
    (In directions 1 atom1 <losses> a lone pair, in direction 2, atom1 <gains> a lone pair)
    """
#     cython.declare(paths=list, atom2=Atom, atom3=Atom, bond12=Bond, bond23=Bond)

    paths = []
    for atom2, bond12 in atom1.edges.items():
        if (not (atom1.radicalElectrons and atom2.radicalElectrons)  # not allowing both atom1, atom2 to be radicals
                and atom2.isNonHydrogen()):  # don't bother with hydrogen atoms
            if bond12.isSingle() or bond12.isDouble():
                # Find paths in the direction <increasing> the bond order
                # atom1 must posses at least one lone pair to loose it
                if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [1, 2, 3])
                        or (atom1.isOxygen() and atom1.lonePairs in [2, 3])  # not allowing O p0
                        and (atom2.radicalElectrons or atom2.charge > 0)):  # to increase the bond order atom2 must
                        # either be a radical or have a positive charge
                    paths.append([atom1, atom2, bond12, 1])  # direction = 1
            if bond12.isDouble() or bond12.isTriple():
                # Find paths in the direction <decreasing> the bond order
                # atom1 gains a lone pair, hence cannot have more than two lone pairs
                if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [0, 1, 2])
                        or (atom1.isOxygen() and atom1.lonePairs in [1, 2])):  # not allowing O p0
                    paths.append([atom1, atom2, bond12, 2])  # direction = 2
#     print "LP-MP paths: {0}".format(paths)
    return paths


def find_N5ddc_N5tc_delocalization_paths(atom1):
    """
    Find all the resonance structures of nitrogen atoms with two double bonds (atomType N5ddc)
    and nitrogen atoms with one triple and one single bond (atomType N5tc).
    Examples:
    - N2O (N#[N+][O-] <-> [N-]=[N+]=O)
    - Azide (N#[N+][NH-] <-> [N-]=[N+]=N <-> [N-2][N+]#[NH+])
    - N#N group on sulfur (O[S-](O)[N+]#N <-> OS(O)=[N+]=[N-] <-> O[S+](O)#[N+][N-2])
    In this transition atom1 is the middle N+ (N5ddc or N5tc)
    """
#     cython.declare(paths=list, atom2=Atom, atom3=Atom, bond12=Bond, bond23=Bond)

    paths = []
    for atom2, bond12 in atom1.edges.items():
        if atom2.isNOS():
            for atom3, bond23 in atom1.edges.items():
                if atom2 is not atom3:
                    # Find this transitions from the N5tc side.
                    # atom2 is single-bonded to atom1 and looses a lone pair
                    # atom3 is triple-bonded to atom1 and gains a lone pair (atom3 cannot be oxygen)
                    # bond12 is incremented, bond23 is decremented
                    if (((atom2.isNitrogen() and atom2.lonePairs in [2, 3])
                            or (atom2.isOxygen() and atom2.lonePairs == 3)
                            or (atom2.isSulfur() and atom2.lonePairs in [1, 2, 3]))
                            and ((atom3.isNitrogen() or atom3.isSulfur()) and atom3.lonePairs in [0, 1])
                            and bond12.isSingle() and bond23.isTriple()):
                        paths.append([atom1, atom2, atom3, bond12, bond23])
                    # Find this transitions from the N5ddc side.
                    # atom2 looses a lone pair and ends up with a triple bond to atom1 (atom2 cannot be oxygen)
                    # atom3 gains a lone pair and ends up with a single bond to atom1
                    # bond12 is incremented, bond23 is decremented (same actions as above under these definitions)
                    elif (((atom2.isNitrogen() or atom2.isSulfur()) and atom2.lonePairs in [1, 2])
                            and ((atom3.isNitrogen() and atom3.lonePairs in [1, 2])
                            or (atom3.isOxygen() and atom3.lonePairs == 2)
                            or (atom3.isSulfur() and atom3.lonePairs in [0, 1, 2]))
                            and bond12.isDouble() and bond23.isDouble()):
                        paths.append([atom1, atom2, atom3, bond12, bond23])
#     print "N5 paths: {0}".format(paths)
    return paths


# Resonance:

def generate_ally_delocalization_resonance_structures(mol):
    """
    Generate all of the resonance structures formed by one allyl radical shift.

    Biradicals on a single atom are not supported.
    """
#     cython.declare(structures=list, paths=list, index=cython.int, structure=Molecule)
#     cython.declare(atom=Atom, atom1=Atom, atom2=Atom, atom3=Atom, bond12=Bond, bond23=Bond)
#     cython.declare(v1=Vertex, v2=Vertex)

    structures = []
    if mol.isRadical():  # Iterate over radicals in structure
        for atom in mol.vertices:
            paths = find_allyl_delocalization_paths(atom)
            for atom1, atom2, atom3, bond12, bond23 in paths:
                # Adjust to (potentially) new resonance structure
                atom1.decrementRadical()
                atom3.incrementRadical()
                bond12.incrementOrder()
                bond23.decrementOrder()
                # Make a copy of isomer
                structure = mol.copy(deep=True)
                # Also copy the connectivity values, since they are the same
                # for all resonance structures
                for index in xrange(len(mol.vertices)):
                    v1 = mol.vertices[index]
                    v2 = structure.vertices[index]
                    v2.connectivity1 = v1.connectivity1
                    v2.connectivity2 = v1.connectivity2
                    v2.connectivity3 = v1.connectivity3
                    v2.sortingLabel = v1.sortingLabel
                # Restore current structure
                atom1.incrementRadical()
                atom3.decrementRadical()
                bond12.decrementOrder()
                bond23.incrementOrder()
                try:
                    structure.updateAtomTypes(logSpecies=False)
                except:
                    pass  # Don't append resonance structure if it creates an undefined atomType
                else:
#                     print "added allyl:"
#                     display(structure)
                    structures.append(structure)
    return structures


def generate_lone_pair_radical_resonance_structures(mol):
    """
    Generate all of the resonance structures formed by lone electron pair - radical shifts.
    These resonance transformations do not involve changing bond orders.
    NO2 example: O=[:N]-[::O.] <=> O=[N.+]-[:::O-]
    (where ':' denotes a lone pair, '.' denotes a radical, '-' not in [] denotes a single bond, '-'/'+' denote charge)
    """
#     cython.declare(structures=list, paths=list, index=cython.int, structure=Molecule)
#     cython.declare(atom=Atom, atom1=Atom, atom2=Atom)
#     cython.declare(v1=Vertex, v2=Vertex)

    structures = []
    if mol.isRadical():  # Iterate over radicals in structure
        for atom in mol.vertices:
            if atom.isNOS():
                paths = find_lone_pair_radical_delocalization_paths(atom)
                for atom1, atom2 in paths:
                    # Adjust to (potentially) new resonance structure
                    atom1.decrementRadical()
                    atom1.incrementLonePairs()
                    atom1.updateCharge()
                    atom2.incrementRadical()
                    atom2.decrementLonePairs()
                    atom2.updateCharge()
                    # Make a copy of structure
                    structure = mol.copy(deep=True)
                    # Also copy the connectivity values, since they are the same
                    # for all resonance structures
                    for index in xrange(len(mol.vertices)):
                        v1 = mol.vertices[index]
                        v2 = structure.vertices[index]
                        v2.connectivity1 = v1.connectivity1
                        v2.connectivity2 = v1.connectivity2
                        v2.connectivity3 = v1.connectivity3
                        v2.sortingLabel = v1.sortingLabel
                    # Restore current structure
                    atom1.incrementRadical()
                    atom1.decrementLonePairs()
                    atom1.updateCharge()
                    atom2.decrementRadical()
                    atom2.incrementLonePairs()
                    atom2.updateCharge()
                    try:
                        structure.updateAtomTypes(logSpecies=False)
                    except:
                        pass  # Don't append resonance structure if it creates an undefined atomType
                    else:
#                         print "added LP-rad:"
#                         display(structure)
                        structures.append(structure)
    return structures


def generate_lone_pair_multiple_bond_resonance_structures(mol):
    """
    Generate all of the resonance structures formed by lone electron pair - multiple bond shifts.
    This function deals with TWO very similar resonance structures, depending whether or not a radical is present:
    - rad N/S/O multiple bond <=> non-radical N/S/O w/ added lonePair and decremented bond (rad gain on other atom)
    - non-rad N/S/O multiple bond <=> non-radical N/S/O w/ added lonePair and decremented bond
    Two simple examples are [N]=C and N=C, respectively:
    - [:N.]=[CH2] <=> [::N]-[.CH2]
    - [:NH]=[CH2] <=> [::NH-]-[CH2+]
    (where ':' denotes a lone pair, '.' denotes a radical, '-' not in [] denotes a single bond, '-'/'+' denote charge)
    Although these are different resonance transformations, their algorithm is identical
    except for loosing/gaining a radical if a radical exists.
    Here atom1 refers to the N/S/O atom, atom 2 refers to the any R!H (atom2's lonePairs aren't affected)
    """
#     cython.declare(structures=list, paths=list, index=cython.int, structure=Molecule, direction=cython.int)
#     cython.declare(atom=Atom, atom1=Atom, atom2=Atom, bond12=Bond)
#     cython.declare(v1=Vertex, v2=Vertex)

    structures = []
    for atom in mol.vertices:
        if atom.isNOS():
            paths = find_lone_pair_multiple_bond_delocalization_paths(atom)
            for atom1, atom2, bond12, direction in paths:
                if atom1.radicalElectrons:  # the radical could be on either atom
                    atom1.decrementRadical()
                    atom2.incrementRadical()
                elif atom2.radicalElectrons:
                    atom2.decrementRadical()
                    atom1.incrementRadical()
                if direction == 1:  # The direction <increasing> the bond order
                    atom1.decrementLonePairs()
                    bond12.incrementOrder()
                elif direction == 2:  # The direction <decreasing> the bond order
                    atom1.incrementLonePairs()
                    bond12.decrementOrder()
                atom1.updateCharge()
                atom2.updateCharge()
                # Make a copy of structure
                structure = mol.copy(deep=True)
                # Also copy the connectivity values, since they are the same
                # for all resonance structures
                for index in xrange(len(mol.vertices)):
                    v1 = mol.vertices[index]
                    v2 = structure.vertices[index]
                    v2.connectivity1 = v1.connectivity1
                    v2.connectivity2 = v1.connectivity2
                    v2.connectivity3 = v1.connectivity3
                    v2.sortingLabel = v1.sortingLabel
                # Restore current structure
                if atom1.radicalElectrons:  # the radical could be on either atom
                    atom1.decrementRadical()
                    atom2.incrementRadical()
                elif atom2.radicalElectrons:
                    atom2.decrementRadical()
                    atom1.incrementRadical()
                if direction == 1:  # The direction <increasing> the bond order
                    atom1.incrementLonePairs()
                    bond12.decrementOrder()
                elif direction == 2:  # The direction <decreasing> the bond order
                    atom1.decrementLonePairs()
                    bond12.incrementOrder()
                atom1.updateCharge()
                atom2.updateCharge()
                try:
                    structure.updateAtomTypes(logSpecies=False)
                except:
                    pass  # Don't append resonance structure if it creates an undefined atomType
                else:
#                     print "added LP-MB:"
#                     display(structure)
                    structures.append(structure)
    return structures


def generate_N5ddc_N5tc_resonance_structures(mol):
    """
    Generate all of the resonance structures formed by shifts between N5dd and N5ts.
    """
#     cython.declare(structures=list, paths=list, index=cython.int, structure=Molecule)
#     cython.declare(atom=Atom, atom1=Atom, atom2=Atom, atom3=Atom, bond12=Bond, bond23=Bond)
#     cython.declare(v1=Vertex, v2=Vertex)
    
    structures = []
    for atom in mol.vertices:
        if atom.atomType.label in ['N5ddc','N5tc'] and atom.radicalElectrons == 0:
            paths = find_N5ddc_N5tc_delocalization_paths(atom)
            for atom1, atom2, atom3, bond12, bond23 in paths:
                atom2.decrementLonePairs()
                atom3.incrementLonePairs()
                bond12.incrementOrder()
                bond23.decrementOrder()
                atom1.updateCharge()
                atom2.updateCharge()
                atom3.updateCharge()
                # Make a copy of structure
                structure = mol.copy(deep=True)
                # Also copy the connectivity values, since they are the same
                # for all resonance structures
                for index in xrange(len(mol.vertices)):
                    v1 = mol.vertices[index]
                    v2 = structure.vertices[index]
                    v2.connectivity1 = v1.connectivity1
                    v2.connectivity2 = v1.connectivity2
                    v2.connectivity3 = v1.connectivity3
                    v2.sortingLabel = v1.sortingLabel
                # Restore current structure
                atom2.incrementLonePairs()
                atom3.decrementLonePairs()
                bond12.decrementOrder()
                bond23.incrementOrder()
                atom1.updateCharge()
                atom2.updateCharge()
                atom3.updateCharge()
                try:
                    structure.updateAtomTypes(logSpecies=False)
                except:
                    pass  # Don't append resonance structure if it creates an undefined atomType
                else:
#                     print "added N5:"
#                     display(structure)
                    structures.append(structure)
    return structures


def generateResonanceStructures(mol, keepIsomorphic=False):
    
    molList = [mol]
    
    index = 0
#     while index < len(molList):
    while index < len(molList) and index < 1000:
        molecule = molList[index]
        newMolList = []

        newMolList += generate_ally_delocalization_resonance_structures(molecule)
        newMolList += generate_lone_pair_radical_resonance_structures(molecule)
        newMolList += generate_lone_pair_multiple_bond_resonance_structures(molecule)
        newMolList += generate_N5ddc_N5tc_resonance_structures(molecule)

        for newMol in newMolList:
            # Append to isomer list if unique
            for mol in molList:
                if not keepIsomorphic and mol.isIsomorphic(newMol):
                    break
                elif keepIsomorphic and mol.isIdentical(newMol):
                    break
            else:
                molList.append(newMol)
                print "index = {0}:".format(index)
                display(newMol)

        # Move to next resonance isomer
        index += 1

    return molList

In [ ]:
# debug:
%pdb

from rmgpy.molecule import Molecule

adj="""multiplicity 3
1 C u1 p0 c0 {2,S} {3,S} {4,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 S u0 p2 c0 {1,S} {5,S}
5 C u0 p0 c0 {4,S} {6,S} {7,D}
6 S u1 p2 c0 {5,S}
7 O u0 p2 c0 {5,D}"""

mol = Molecule().fromAdjacencyList(adj)

print "original molecule:"
display(mol)

molList = generateResonanceStructures(mol, keepIsomorphic=False)

indexList = []
for index in xrange(len(molList)):
    if mol.isIsomorphic(molList[index]):
        indexList.append(index)
    if molList[index].isIsomorphic(mol):
        indexList.append(index)
print indexList

In [ ]:
list1 = [0,1,2,3,4,5,6,7,8,9]
print list1[:5]
print list1[5:]

In [ ]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species
from rmgpy import settings

import os.path
import logging
import codecs
from copy import deepcopy
from IPython.display import display

from rmgpy.constraints import failsSpeciesConstraints
from rmgpy.data.base import Database, Entry, LogicNode, LogicOr, ForbiddenStructures,\
                            getAllCombinations
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius
from rmgpy.molecule import Bond, GroupBond, Group, Molecule
from rmgpy.species import Species

from rmgpy.data.kinetics.common import saveEntry
from rmgpy.data.kinetics.depository import KineticsDepository
from rmgpy.data.kinetics.groups import KineticsGroups
from rmgpy.data.kinetics.rules import KineticsRules
from rmgpy.exceptions import InvalidActionError, ReactionPairsError, KineticsError,\
                             UndeterminableKineticsError, ForbiddenStructureException,\
                             KekulizationError, ActionError

In [ ]:
adj1="""multiplicity 3
1 C u1 p0 c0 {2,S} {3,S} {4,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 S u0 p2 c0 {1,S} {5,S}
5 C u0 p0 c0 {4,S} {6,S} {7,D}
6 S u1 p2 c0 {5,S}
7 O u0 p2 c0 {5,D}"""

mol1 = Molecule().fromAdjacencyList(adj1)
display(mol)
print mol.toAdjacencyList()

for atom in mol.vertices:
    if atom.isNonHydrogen():
        print "atomType for {0} is {1}".format(atom,atom.atomType.label)
        

display(mol1)
print mol1.toAdjacencyList()

for atom in mol1.vertices:
    if atom.isNonHydrogen():
        print "atomType for {0} is {1}".format(atom,atom.atomType.label)

In [ ]:
adj2="""multiplicity 3
1 C u1 p0 c0 {2,S} {3,S} {4,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 S u0 p2 c0 {1,S} {5,S}
5 C u0 p0 c0 {4,S} {6,S} {7,D}
6 S u1 p2 c0 {5,S}
7 O u0 p2 c0 {5,D}"""

mol1 = Molecule().fromAdjacencyList(adj2)
display(mol1)
        
        
paths = []
for atom1 in mol1.vertices:
    for atom2, bond12 in atom1.edges.items():
        if (not (atom1.radicalElectrons and atom2.radicalElectrons)  # not allowing both atom1, atom2 to be radicals
            and atom2.isNonHydrogen()):  # don't bother with hydrogen atoms
            if bond12.isSingle() or bond12.isDouble():
                # Find paths in the direction <increasing> the bond order
                # atom1 must posses at least one lone pair to loose it
                if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [1, 2, 3])
                    or (atom1.isOxygen() and atom1.lonePairs in [2, 3])  # not allowing O p0
                    and (atom2.radicalElectrons or atom2.charge > 0)):  # to increase the bond order atom2 must
                    # either be a radical or have a positive charge
                    paths.append([atom1, atom2, bond12, 1])  # direction = 1
                    print "appending dir 1 for atom1 {0}, atom2 {1}".format(atom1,atom2)
            if bond12.isDouble() or bond12.isTriple():
                # Find paths in the direction <decreasing> the bond order
                # atom1 gains a lone pair, hence cannot have more than two lone pairs
                if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [0, 1, 2])
                    or (atom1.isOxygen() and atom1.lonePairs in [1, 2])):  # not allowing O p0
                    paths.append([atom1, atom2, bond12, 2])  # direction = 2
                    print "appending dir 2 for atom1 {0}, atom2 {1}".format(atom1,atom2)

                    
isomers = []
for atom1, atom2, bond12, direction in paths:
    if atom1.radicalElectrons:  # the radical could be on either atom
        atom1.decrementRadical()
        atom2.incrementRadical()
    elif atom2.radicalElectrons:
        atom2.decrementRadical()
        atom1.incrementRadical()
    if direction == 1:  # The direction <increasing> the bond order
        atom1.decrementLonePairs()
        bond12.incrementOrder()
    elif direction == 2:  # The direction <decreasing> the bond order
        atom1.incrementLonePairs()
        bond12.decrementOrder()
    atom1.updateCharge()
    atom2.updateCharge()
    # Make a copy of isomer
    isomer = mol.copy(deep=True)
    # Also copy the connectivity values, since they are the same
    # for all resonance forms
    for index in range(len(mol.vertices)):
        v1 = mol.vertices[index]
        v2 = isomer.vertices[index]
        v2.connectivity1 = v1.connectivity1
        v2.connectivity2 = v1.connectivity2
        v2.connectivity3 = v1.connectivity3
        v2.sortingLabel = v1.sortingLabel
    # Restore current isomer
    if atom1.radicalElectrons:  # the radical could be on either atom
        atom1.decrementRadical()
        atom2.incrementRadical()
    elif atom2.radicalElectrons:
        atom2.decrementRadical()
        atom1.incrementRadical()
    if direction == 1:  # The direction <increasing> the bond order
        atom1.incrementLonePairs()
        bond12.decrementOrder()
        print "direction 1"
    elif direction == 2:  # The direction <decreasing> the bond order
        atom1.decrementLonePairs()
        bond12.incrementOrder()
        print "direction 2"
    atom1.updateCharge()
    atom2.updateCharge()
    try:
        isomer.updateAtomTypes(logSpecies=False)
    except:
        print "unsucsessful try"
        pass  # Don't append resonance structure if it creates an undefined atomType
    else:
        print "sucsess!"
        isomers.append(isomer)


for isomer in isomers:
    display(isomer)



In [ ]:
print mol.isIsomorphic(mol1)

In [ ]:
display(mol)
print mol.toAdjacencyList()
display(mol1)
print mol1.toAdjacencyList()

In [ ]:
database = RMGDatabase()
database.load(
            path = '/home/alongd/Code/RMG-database/input/',
            thermoLibraries = '',
            transportLibraries = '',
            reactionLibraries = '',
            seedMechanisms = '',
            kineticsFamilies = ['CO_Disproportionation'],
            kineticsDepositories = ['training'],
            depository = False, # Don't bother loading the depository information, as we don't use it
)

In [ ]:
for self in database.kinetics.families.values():
    depository = self.getTrainingDepository()
index = max([e.index for e in self.rules.getEntries()] or [0]) + 1

entries = depository.entries.values()
entries.sort(key=lambda x: x.index)
reverse_entries = []

for entry in entries:
    reaction = entry.item
    
    if True:
    
 #   def getReactionTemplate(self, reaction):
        """
        For a given `reaction` with properly-labeled :class:`Molecule` objects
        as the reactants, determine the most specific nodes in the tree that
        describe the reaction.
        """
    
        # Get forward reaction template and remove any duplicates
        forwardTemplate = self.top[:]

        temporary = []
        symmetricTree = False
        for entry in forwardTemplate:
            if entry not in temporary:
                temporary.append(entry)
            else:
                # duplicate node found at top of tree
                # eg. R_recombination: ['Y_rad', 'Y_rad']
                assert len(forwardTemplate)==2 , 'Can currently only do symmetric trees with nothing else in them'
                symmetricTree = True
        forwardTemplate = temporary

        # Descend reactant trees as far as possible
        template = []
        for entry in forwardTemplate:
            # entry is a top-level node that should be matched
            group = entry.item

            # Identify the atom labels in a group if it is not a logical node
            atomList = []
            if not isinstance(entry.item, LogicNode):
                atomList = group.getLabeledAtoms()

            for reactant in reaction.reactants:
                if isinstance(reactant, Species):
                    reactant = reactant.molecule[0]
                # Match labeled atoms
                # Check that this reactant has each of the atom labels in this group. If it is a LogicNode,
                # the atomList is empty and it will proceed directly to the descendTree step.
                if not all([reactant.containsLabeledAtom(label) for label in atomList]):
                    continue # don't try to match this structure - the atoms aren't there!
                # Match structures
                atoms = reactant.getLabeledAtoms()
                # Descend the tree, making sure to match atomlabels exactly using strict = True
                matched_node = self.descendTree(reactant, atoms, root=entry, strict=True)
                if matched_node is not None:
                    template.append(matched_node)
                #else:
                #    logging.warning("Couldn't find match for {0} in {1}".format(entry,atomList))
                #    logging.warning(reactant.toAdjacencyList())

        # Get fresh templates (with duplicate nodes back in)
        forwardTemplate = self.top[:]
        if self.label.lower().startswith('r_recombination'):
            forwardTemplate.append(forwardTemplate[0])

        # Check that we were able to match the template.
        # template is a list of the actual matched nodes
        # forwardTemplate is a list of the top level nodes that should be matched
        if len(template) != len(forwardTemplate):
#            print 'len(template):', len(template)
#            print 'len(forwardTemplate):', len(forwardTemplate)
            msg = 'Unable to find matching template for reaction {0} in reaction family {1}.'.format(str(reaction), str(self)) 
            msg += 'Trying to match {0} but matched {1}'.format(str(forwardTemplate),str(template))
#            print 'reactants'
#            for reactant in reaction.reactants:
#                print reactant.toAdjacencyList() + '\n'
#            print 'products'
#            for product in reaction.products:
#                print product.toAdjacencyList() + '\n'
            raise UndeterminableKineticsError(reaction, message=msg)

        print "template: ", template
    
    

In [ ]:
self

In [ ]:
import rmgpy.molecule as molecule

adj="""multiplicity 2
1    O u0 p2 c0 {2,D}
2    N u0 p1 c0 {1,D} {3,S}
3 *1 O u1 p2 c0 {2,S}"""

reactant = molecule.Molecule().fromAdjacencyList(adj)

atoms = reactant.getLabeledAtoms()

atoms